In [18]:
import pandas as pd
pd.set_option('max_colwidth', 300)
import seaborn as sns
import numpy as np
import random
import csv
sns.set()

In [19]:
fh = list(open('./top_k_10_temp_1.txt', 'rt'))
random.Random(119).shuffle(fh)

data = []
for line in fh:
    line = line.split('\t')
    
    gender = line[0]
    q = line[1].split('<|endoftext|>')[0].strip().strip('!')
    response = line[1].split('<|endoftext|>')[1].strip()
    
    # checks 
    if len(response) < 2:
        continue
    if len(q) < 20:
        continue
    
    data.append((gender, q, response))

In [20]:
data = data[:600]

In [21]:
# split into twelves
header = [ ['g%d' %i, 'q%d' %i, 'r%d' %i] for i in range(1,13)]
header = sum(header, [])

flat_data = []
for i in range(0, len(data), 12):
    chunk = data[i:i+12]
    row = sum(chunk, (),)
    flat_data.append(row)

In [22]:
# write to file
with open('./mturk_600_topk_10_temp_1.csv', 'wt') as f:
    fh = csv.writer(f)
    fh.writerow(header)
    for line in flat_data:
        fh.writerow(line)
print('Done.')

Done.


### Annotation

In [34]:
df = pd.DataFrame(data)
df.columns = ['index', 'q', 'r']
sample = df.sample(100)

In [47]:
sample.head()

,index,q,r
228,M,"If you arent setting goals and trying to reach them, what exactly are you doing?","I was wondering if anyone had any suggestions. I've been looking for a better program, and this is the only one that"
382,M,"Which... okay, wait, wasnt Colleens dojo struggling, and isnt The Hand some sort of massive entity?",The hand is an entity but is a massive force of nature.
232,M,And the last ten years of that series?,That was the last season.
566,W,"What better way to make everyone think the victim is crazy than to be the nicest, gentlest guy they know?","He's a good guy, he is"
137,W,What does your husbands family usually do for holidays?,We do the most amazing things!


In [36]:
from pigeon import annotate
annotations = annotate(
  ['Q:%s ||| A:%s' % (row['q'], row['r']) for i, row in sample.iterrows()],
  options=['fully_sat', 'some_related_info', 'some_unrelated_info', 'non_response', 'does_not_make_sense']
)

HTML(value='0 examples annotated, 101 examples left')

Output()

Annotation done.


In [50]:
'''
with open('question_annotations.txt', 'wt') as fh:
    for (i, j), gender in zip(annotations, sample['index']):
        i = i.split(' ||| ')
        fh.write('%s\t%s\t%s\t%s\n' % (gender, i[0][2:], i[1][2:], j))
'''

"\nwith open('question_annotations.txt', 'wt') as fh:\n    for (i, j), gender in zip(annotations, sample['index']):\n        i = i.split(' ||| ')\n        fh.write('%s\t%s\t%s\t%s\n' % (gender, i[0][2:], i[1][2:], j))\n"

### Analysis

In [54]:
annotated = pd.read_csv('question_annotations.txt', sep='\t', header=None)
annotated.head()

,0,1,2,3
0,M,"If you arent setting goals and trying to reach them, what exactly are you doing?","I was wondering if anyone had any suggestions. I've been looking for a better program, and this is the only one that",some_unrelated_info
1,M,"Which... okay, wait, wasnt Colleens dojo struggling, and isnt The Hand some sort of massive entity?",The hand is an entity but is a massive force of nature.,fully_sat
2,M,And the last ten years of that series?,That was the last season.,does_not_make_sense
3,W,"What better way to make everyone think the victim is crazy than to be the nicest, gentlest guy they know?","He's a good guy, he is",some_related_info
4,W,What does your husbands family usually do for holidays?,We do the most amazing things!,fully_sat


In [59]:
for i, group in annotated.groupby(0):
    print(i)
    print(group[3].value_counts())

M
some_related_info      15
fully_sat              14
does_not_make_sense    10
some_unrelated_info     5
non_response            5
Name: 3, dtype: int64
W
fully_sat              15
some_related_info      12
does_not_make_sense    11
some_unrelated_info     9
non_response            4
Name: 3, dtype: int64


In [65]:
for i, group in annotated.groupby(3):
    print('---')
    print(i)
    for i, row in group[[1,2]].iterrows():
        print('Q:%s\nA:%s\n' % (row[1], row[2]))

---
does_not_make_sense
Q:And the last ten years of that series?
A:That was the last season.

Q:I had raved about it to my husband and he was like,  wtf is this youre making me watch?
A:Haha I'm glad your wife got the message.

Q:I think they generally make it pretty clear, dont they?
A:I think that's a very fair rule, I just don't think it's a fair rule if you are the one who is supposed to

Q:Using your example of space exploration, do you know why a company like SpaceX can build rockets cheaper and more quickly?
A:Yes, it's the same reason why the

Q:I dont know if that helps at all?
A:I think you meant I don't know but I can't help with what I don't know. :

Q:Ill spell it out nice and clear:  > So no one?
A:I'm on it.

Q:Its just kind of funny to see how much they DONT get it - are they socially inept and cant pick up on obvious stop signals!?
A:They're just too busy laughing at people who

Q:So its worse when it happens to a woman?
A:Yes, it's worse to have your child with a woma